In [2]:
from utils import db_connection
from utils import s3_connection

Following query executed, closing connection | SELECT * FROM order_schema.order_with_cost
1
quantrix-create-bucket-test2 does not exist


In [16]:
# 1. Create  a file:
# col:
# product_name varchar(20),
# manufacturingcost decimal(5,2)

# For each product build record using python script.
# 'iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512',
# 'galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340'
# 'Nk320',''Nk400',''Nk500'

# File example:
# iphon11,220
# iphone12,250
# iphone13,290

# Generate order information from gen_dummy_data.py

# Create CSV for product cost

import pandas as pd


product_costs = {'iphone11': 800, 'iphone12': 900, 'iphone13': 950, 'iphoneSE': 700, 'IpadMax': 500, 'IpadMini': 400,
				 'laptop256': 1400, 'Macbook512': 1600, 'galaxy10': 800, 'galaxy11': 850, 'galaxy12': 875,
				 'galaxy13': 900, 'watch320': 249, 'watch340': 300, 'Nk320': 600, 'Nk400': 700, 'Nk500': 750}
result = []
for k, v in product_costs.items():
	result.append((k, v))

df = pd.DataFrame(result, columns=['product', 'product_cost'])
df.to_csv('product_costs1.csv', index=False)


# create a table in redshift also.-- table name product_cost
from utils import db_connection

db_connection.redshift_query('''Create SCHEMA IF NOT EXISTS order_schema;''')

db_connection.redshift_query('''Create table if not exists order_schema.product_costs 
                                (product varchar(20),product_cost varchar(20));''')


Following query executed, closing connection | Create SCHEMA IF NOT EXISTS order_schema;
Following query executed, closing connection | Create table if not exists order_schema.product_costs 
                                (product varchar(20),product_cost varchar(20));


1

In [5]:
# 2. Load data into s3 in separate folder (order data must be created earlier in class project 3).
from utils import s3_connection

s3_connection.s3_create_bucket('quantrix-order-data', 'us-east-2')

# Done in AWS CLI
# C:\Users\user>aws s3 cp "C:\FILEPATH\order_data_20230113.csv" s3://quantrix-order-data/raw/

# C:\Users\user>aws s3 cp "C:\FILEPATH\product_costs.csv" s3://quantrix-order-data

quantrix-order-data bucket already exists


-1

In [17]:
# 3. Use copy command  to load data into redshift by.

# create table for the order data to populate | order_data and product_cost

db_connection.redshift_query('''Create table if not exists order_schema.order_data (
                                   order_id int,
                                   brand_name varchar(20),
                                   product_name varchar(20),
                                   sales_amount decimal(9,2),
                                   sales_date date
                                )''')

db_connection.redshift_query('''copy order_schema.order_data from 's3://quantrix-order-data/raw/order_data_20230113.csv' 
                                iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
                                delimiter ','
                                ignoreheader 1
                                ''')

db_connection.redshift_query('''copy order_schema.product_costs from 's3://quantrix-order-data/product_costs.csv' 
                                iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
                                delimiter ','
                                ignoreheader 1
                                ''')



Following query executed, closing connection | Create table if not exists order_schema.order_data (
                                   order_id int,
                                   brand_name varchar(20),
                                   product_name varchar(20),
                                   sales_amount decimal(9,2),
                                   sales_date date
                                )
Following query executed, closing connection | copy order_schema.order_data from 's3://quantrix-order-data/raw/order_data_20230113.csv' 
                                iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
                                delimiter ','
                                ignoreheader 1
                                
Following query executed, closing connection | copy order_schema.product_costs from 's3://quantrix-order-data/product_costs.csv' 
                                iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
                               

1

In [18]:
# 4  create a view by joining order and product_cost.

db_connection.redshift_query('''CREATE VIEW order_schema.order_with_cost as(
    SELECT od.*, pc.product_cost as product_cost From order_schema.order_data od
    JOIN order_schema.product_costs pc
    ON od.product_name = pc.product
    ) WITH NO SCHEMA BINDING
    ''')

Following query executed, closing connection | CREATE VIEW order_schema.order_with_cost as(
    SELECT od.*, pc.product_cost as product_cost From order_schema.order_data od
    JOIN order_schema.product_costs pc
    ON od.product_name = pc.product
    ) WITH NO SCHEMA BINDING
    


1

In [26]:
# 5. Create a bucket using boto3 and then list the buckets (from the metadata). make sure the bucket exists else fail.

from utils import s3_connection

s3_connection.print_functions()

s3_connection.s3_create_bucket('quantrix-order-transformation', 'us-east-2')

S3 Commands: 
	list_s3_buckets() | returns list of existing buckets in S3 
	s3_bucket_exists(bucket_name: string) | return bool showing existence of bucket in s3 
	s3_create_bucket(new_bucket_name: string, region_name: string *optional*) | creates new bucket 
	s3_delete_bucket(bucket_name: string) | deletes selected bucket if exists
quantrix-order-transformation bucket already exists


-1

In [27]:
# 6. Unload data from view into s3 bucket (this has to be separate bucket then created earlier) 
# buckets/transformation/vw_name/    (this has ot be a csv file with header)

db_connection.redshift_query('''UNLOAD ('SELECT * from order_schema.order_with_cost')
    to 's3://quantrix-order-transformation/unload_results/'
    iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
    csv
    header''')

Following query executed, closing connection | UNLOAD ('SELECT * from order_schema.order_with_cost')
    to 's3://quantrix-order-transformation/unload_results/'
    iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
    csv
    header


1

In [29]:
# 7. Create a new view by taking all the records where you have profit. Profit is sales amount -manufacturingcost. 
# So add all the fields from order table and cost from product_cost.

db_connection.redshift_query('''CREATE VIEW order_schema.orders_with_profit as(
    SELECT * From order_schema.order_with_cost
    WHERE (sales_amount - product_cost) > 0
    ) WITH NO SCHEMA BINDING''')

Following query executed, closing connection | CREATE VIEW order_schema.orders_with_profit as(
    SELECT * From order_schema.order_with_cost
    WHERE (sales_amount - product_cost) > 0
    ) WITH NO SCHEMA BINDING


1

In [30]:
# 8. Create col as profit_group the values are [if profit percenatge is more 10% the "Bumper profit", 
#                                               0 to 10% then "Marginal profit",
#                                               -5% to 0% then "Loss making",
#                                               else "Bumper Loss"

db_connection.redshift_query('''CREATE VIEW order_schema.orders_with_profit_group as(
    SELECT *, CASE WHEN (sales_amount/product_cost) > 1.1 THEN 'Bumper profit'
				WHEN (sales_amount/product_cost) >= 1 THEN 'Marginal profit'
                WHEN (sales_amount/product_cost) >= .95 THEN 'Loss making'
				ELSE 'Bumper Loss' END as profit_group
	FROM order_schema.order_with_cost
) WITH NO SCHEMA BINDING''')

Following query executed, closing connection | CREATE VIEW order_schema.orders_with_profit_group as(
    SELECT *, CASE WHEN (sales_amount/product_cost) > 1.1 THEN 'Bumper profit'
				WHEN (sales_amount/product_cost) >= 1 THEN 'Marginal profit'
                WHEN (sales_amount/product_cost) >= .95 THEN 'Loss making'
				ELSE 'Bumper Loss' END as profit_group
	FROM order_schema.order_with_cost
) WITH NO SCHEMA BINDING


1

In [31]:
# 9. Unload the view into a s3 file.

db_connection.redshift_query('''UNLOAD ('SELECT * from order_schema.orders_with_profit_group')
    to 's3://quantrix-order-data/unload_with_profit_group/'
    iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
    csv
    header''')

Following query executed, closing connection | UNLOAD ('SELECT * from order_schema.orders_with_profit_group')
    to 's3://quantrix-order-data/unload_with_profit_group/'
    iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
    csv
    header


1

In [3]:
# 10. Create another view to show total profit by each brand_name,year-mth cd (ex:202201,202203). 
# Do a rank by the total profit (desc) and amount of product sold (Asc)

db_connection.redshift_query('''CREATE VIEW order_schema.orders_profit_by_yearmonth_brand as(
    SELECT TO_CHAR(sales_date, 'YYYY-MM') AS year_month, brand_name, SUM(sales_amount-product_cost) as total_profit, RANK() OVER (ORDER BY total_profit DESC) as rank_highest_profit, 
        COUNT(1) as num_products_sold, RANK() Over(ORDER BY num_products_sold ASC) as rank_least_products_sold
    FROM order_schema.order_with_cost
    GROUP BY year_month, brand_name
    ORDER BY year_month, brand_name
    ) WITH NO SCHEMA BINDING''')

Following query executed, closing connection | CREATE VIEW order_schema.orders_profit_by_yearmonth_brand as(
    SELECT TO_CHAR(sales_date, 'YYYY-MM') AS year_month, brand_name, SUM(sales_amount-product_cost) as total_profit, RANK() OVER (ORDER BY total_profit DESC) as rank_highest_profit, 
        COUNT(1) as num_products_sold, RANK() Over(ORDER BY num_products_sold ASC) as rank_least_products_sold
    FROM order_schema.order_with_cost
    GROUP BY year_month, brand_name
    ORDER BY year_month, brand_name
    ) WITH NO SCHEMA BINDING


1

In [33]:
# 11. unload data to a CSV file wtih header and pipe delimited

db_connection.redshift_query('''UNLOAD ('SELECT * from order_schema.orders_profit_by_yearmonth_brand')
    to 's3://quantrix-order-data/unload_profits_by_yearmonth_brand/'
    iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
    delimiter '|'
    csv
    header''')

Following query executed, closing connection | UNLOAD ('SELECT * from order_schema.orders_profit_by_yearmonth_brand')
    to 's3://quantrix-order-data/unload_profits_by_yearmonth_brand/'
    iam_role 'arn:aws:iam::<USER>:role/<ROLE_NAME>'
    delimiter '|'
    csv
    header


1